In [1]:
import numpy as np
import rootutils

root = rootutils.setup_root(search_from=".", indicator=".git")

DATA_DIR = root / "data"

In [2]:
dl_data_path = DATA_DIR / "preproc" / "dl_data.npz"
dl_data = np.load(dl_data_path, allow_pickle=True)

In [3]:
print(dl_data)
print("Keys:", dl_data.files)
for key in dl_data.files:
    print(f"{key}: shape={dl_data[key].shape}, dtype={dl_data[key].dtype}")

NpzFile '/Users/arian/Documents/FHNW/cdl1/CDL1-MChallenge/data/preproc/dl_data.npz' with keys: X, y
Keys: ['X', 'y']
X: shape=(4129, 250, 20), dtype=float32
y: shape=(4129,), dtype=<U8


CNN
LSTM
MLP

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Prepare data
X = dl_data['X']  # shape: (N, T, C) or (N, C, T)
y = dl_data['y']  # shape: (N,)

# If y is object dtype or string, encode to integer labels
if y.dtype.kind in {'U', 'S', 'O'} or not np.issubdtype(y.dtype, np.integer):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(y)
    print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))
else:
    le = None  # No label encoding needed

if X.shape[1] < X.shape[2]:  # (N, T, C) -> (N, C, T)
    X = np.transpose(X, (0, 2, 1))

X = X.astype(np.float32)
y = y.astype(np.int64)
num_classes = len(np.unique(y))

# Convert to torch tensors
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

# Split into train, val, test (e.g., 70/15/15)
N = X.shape[0]
n_train = int(0.7 * N)
n_val = int(0.15 * N)
n_test = N - n_train - n_val
dataset = TensorDataset(X_tensor, y_tensor)
train_set, val_set, test_set = random_split(dataset, [n_train, n_val, n_test], generator=torch.Generator().manual_seed(42))

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

# Define a simple 1D CNN
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(in_channels=X.shape[1], num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
def train_model(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                val_loss += loss.item() * xb.size(0)
                preds = out.argmax(dim=1).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(yb.cpu().numpy())
        val_loss /= len(val_loader.dataset)
        val_acc = accuracy_score(np.concatenate(all_labels), np.concatenate(all_preds))
        print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f} - Val acc: {val_acc:.4f}")

train_model(model, train_loader, val_loader, epochs=20)

# Evaluation on test set
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(yb.cpu().numpy())
y_true = np.concatenate(all_labels)
y_pred = np.concatenate(all_preds)

acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Precision: {prec:.4f}")
print(f"Test Recall: {rec:.4f}")
print(f"Test F1-score: {f1:.4f}")

# If you want to map predictions back to string labels:
if le is not None:
    y_true_labels = le.inverse_transform(y_true)
    y_pred_labels = le.inverse_transform(y_pred)
    print("Example true labels:", y_true_labels[:10])
    print("Example predicted labels:", y_pred_labels[:10])


Label mapping: {np.str_('climbing'): np.int64(0), np.str_('joggen'): np.int64(1), np.str_('sitting'): np.int64(2), np.str_('walking'): np.int64(3)}
Epoch 1/20 - Train loss: 0.7310 - Val loss: 0.4699 - Val acc: 0.8498
Epoch 2/20 - Train loss: 0.4027 - Val loss: 0.3471 - Val acc: 0.8691
Epoch 3/20 - Train loss: 0.3421 - Val loss: 0.3066 - Val acc: 0.8740
Epoch 4/20 - Train loss: 0.3076 - Val loss: 0.2901 - Val acc: 0.8756
Epoch 5/20 - Train loss: 0.2905 - Val loss: 0.2771 - Val acc: 0.8998
Epoch 6/20 - Train loss: 0.2769 - Val loss: 0.2598 - Val acc: 0.8885
Epoch 7/20 - Train loss: 0.2578 - Val loss: 0.2526 - Val acc: 0.9063
Epoch 8/20 - Train loss: 0.2558 - Val loss: 0.2439 - Val acc: 0.9079
Epoch 9/20 - Train loss: 0.2345 - Val loss: 0.2349 - Val acc: 0.9079
Epoch 10/20 - Train loss: 0.2248 - Val loss: 0.2129 - Val acc: 0.9160
Epoch 11/20 - Train loss: 0.2037 - Val loss: 0.2145 - Val acc: 0.9144
Epoch 12/20 - Train loss: 0.1950 - Val loss: 0.2074 - Val acc: 0.9225
Epoch 13/20 - Train l